## Utilizando Regressão Linear para predizer o preço máximo ao consumidor dos medicamentos brasileiros

#### Importando as bibliotecas

In [39]:
import pandas as pd
import numpy as np
import sklearn

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#### Importando os dados

In [2]:
df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')

C:\Users\jose-\AppData\Local\Temp\ipykernel_8920\1792964528.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')


### Capturando as colunas importantes
<b>Tipo de produto (status do produto)</b>: Indica qual se a venda do medicamento necessita de recita.<br>
<b>Tarja</b>: indica se a venda do medicamento necessita de receita, sendo classificado em Tarja preta, Tarja vermelha ou Venda livre.<br>
<b>PF 18% alc</b>: preço de fábrica.<br>
<b>PMC 18% alc</b>: preço máximo ao consumidor (variável alvo).<br>
<br>
<b>Obs:</b> Foram escolhidos os PF e PMC com alíquota de 18%, ao invés das outra porcentagens, pois este é o percentual aplicado na maioria dos estados brasileiros.

In [3]:
# Deixa os nomes das colunas em letras minúsculas
df.rename(columns = {column:column.lower() for column in df.columns}, inplace = True)

# Simplifica nomes das colunas
df.rename(columns = {'tipo de produto (status do produto)': 'tipo_produto',
                    'pf 18% alc': 'pf_18',
                    'pmc 18% alc': 'pmc_18'}, inplace = True)

df = df[['tipo_produto', 'tarja', 'pf_18','pmc_18']]

In [4]:
# Retira linhas com dados nulos

df.dropna(inplace = True)

In [5]:
df.head()

,tipo_produto,tarja,pf_18,pmc_18
0,Similar,Tarja Vermelha (**),44,"60,83"
1,Similar,Tarja Preta (**),"6,15","8,5"
2,-,Tarja Vermelha,"130,95","181,03"
3,Novo,Tarja Vermelha (**),"60,4","83,5"
4,Novo,Tarja Vermelha (**),"60,4","83,5"


### Consertando os dados
Removeremos do dataset as linhas que não contém significado na coluna 'tipo_produto'.<br> 

Na coluna 'tarja' reduziremos à tarjas vermelha, preta e livre.<br>

Na coluna 'pf_18' e 'pmc_18' faremos a conversão do tipo string para o tipo float.<br>

In [6]:
# Corrige dataset a partir da coluna 'tipo_produto'

df = df[df['tipo_produto'].str.contains('-') == False]
df['tipo_produto'].value_counts()

Genérico        8069
Similar         7885
Novo            3835
Específico      1435
Biológico        822
Fitoterápico     389
Name: tipo_produto, dtype: int64

In [7]:
# Simplifica coluna 'tarja'

def simplifica_tarja(tarja):
    
    if 'erm' in tarja:
        return 'Tarja vermelha'
    elif 'ret' in tarja:
        return 'Tarja preta'
    else:
        return 'Sem tarja'
    
df['tarja'] = df['tarja'].apply(simplifica_tarja)
df['tarja'].value_counts()

Tarja vermelha    17765
Sem tarja          3292
Tarja preta        1378
Name: tarja, dtype: int64

In [8]:
# Converte PF e PMC para o tipo float

def converte_float(preco):
    
    if ',' in preco:
        preco = preco.replace(',', '.')
    
    preco = float(preco)
    return preco

df['pf_18'] = df['pf_18'].apply(converte_float)
df['pmc_18'] = df['pmc_18'].apply(converte_float)

In [9]:
# Visão do dataset consertado

df.sample(5)

,tipo_produto,tarja,pf_18,pmc_18
25435,Novo,Tarja vermelha,73.74,101.94
26591,Genérico,Tarja vermelha,22.69,31.37
2919,Novo,Tarja preta,22.32,30.86
23934,Genérico,Tarja vermelha,26.87,37.15
18410,Similar,Tarja vermelha,95.41,131.90


In [10]:
# Cria os dummies para as colunas categóricas tipo_produto e tarja

final_df = pd.get_dummies(df, drop_first = True)
final_df.head()

,pf_18,pmc_18,tipo_produto_Específico,tipo_produto_Fitoterápico,tipo_produto_Genérico,tipo_produto_Novo,tipo_produto_Similar,tarja_Tarja preta,tarja_Tarja vermelha
0,44.00,60.83,0,0,0,0,1,0,1
1,6.15,8.50,0,0,0,0,1,1,0
3,60.40,83.50,0,0,0,1,0,0,1
4,60.40,83.50,0,0,0,1,0,0,1
5,40.61,56.14,0,0,0,1,0,0,1


### Separando os dados de treino e de teste

In [26]:
predict = 'pmc_18'

X = np.array(final_df.drop([predict], 1))
y = np.array(final_df[predict])

C:\Users\jose-\AppData\Local\Temp\ipykernel_8920\3148247265.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(final_df.drop([predict], 1))


In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 37)

### Ajustando (fitting) o modelo de Regressão Linear ao dataset

In [31]:
regressor = linear_model.LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

### Executando as predições e as avaliando 

In [35]:
y_pred = regressor.predict(x_test)

In [42]:
print(f'R² Score: {r2_score(y_pred, y_test)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_pred, y_test)}')
print(f'Mean Squared Error: {mean_squared_error(y_pred, y_test)}')

R² Score: 0.9999999999999557
Mean Absolute Error: 0.0024965595487898193
Mean Squared Error: 8.324417231812215e-06
